In [1]:
# imports
import tensorflow as tf
from random import randint,seed
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
#from keras.callbacks import ModelCheckpoint
from sklearn import datasets
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectFromModel
from keras.models import Sequential
from keras.layers import LSTM, Dense,GRU,RNN,SimpleRNN,Dropout,Activation,Softmax,regularizers
from keras.models import load_model
from keras import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from numpy import array
from scipy.sparse import csr_matrix
from keras.models import Sequential
from keras.layers import LSTM, Dense,GRU,RNN,SimpleRNN,Dropout,RepeatVector,TimeDistributed
from keras.models import load_model
from keras.datasets import imdb
from keras.preprocessing import sequence
from numpy import argmax


from random import seed
from random import randint
from numpy import array
from math import ceil
from math import log10
from math import sqrt





Using TensorFlow backend.


In [2]:

 
# generate lists of random integers and their sum
def random_sum_pairs(n_examples, n_numbers, largest):
    X, y = list(), list()
    for i in range(n_examples):
        in_pattern = [randint(1,largest) for _ in range(n_numbers)]
        out_pattern = sum(in_pattern)
        X.append(in_pattern)
        y.append(out_pattern)
    return X, y
 
# convert data to strings
def to_string(X, y, n_numbers, largest):
    max_length = n_numbers * ceil(log10(largest+1)) + n_numbers - 1
    Xstr = list()
    for pattern in X:
        strp = '+'.join([str(n) for n in pattern])
        strp = ''.join([' ' for _ in range(max_length-len(strp))]) + strp
        Xstr.append(strp)
    max_length = ceil(log10(n_numbers * (largest+1)))
    ystr = list()
    for pattern in y:
        strp = str(pattern)
        strp = ''.join([' ' for _ in range(max_length-len(strp))]) + strp
        ystr.append(strp)
    return Xstr, ystr
 
# integer encode strings
def integer_encode(X, y, alphabet):
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))
    Xenc = list()
    for pattern in X:
        integer_encoded = [char_to_int[char] for char in pattern]
        Xenc.append(integer_encoded)
        yenc = list()
    for pattern in y:
        integer_encoded = [char_to_int[char] for char in pattern]
        yenc.append(integer_encoded)
    return Xenc, yenc
 
# one hot encode
def one_hot_encode(X, y, max_int):
    Xenc = list()
    for seq in X:
        pattern = list()
        for index in seq:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        Xenc.append(pattern)
    yenc = list()
    for seq in y:
        pattern = list()
        for index in seq:
            vector = [0 for _ in range(max_int)]
            vector[index] = 1
            pattern.append(vector)
        yenc.append(pattern)
    return Xenc, yenc





# invert encoding
def invert(seq, alphabet):
    int_to_char = dict((i, c) for i, c in enumerate(alphabet))
    strings = list()
    for pattern in seq:
        string = int_to_char[argmax(pattern)]
        strings.append(string)
    return ''.join(strings)

In [3]:
seed(1)
n_samples = 1
n_numbers = 2
largest = 10
# generate pairs
X, y = random_sum_pairs(n_samples, n_numbers, largest)
print(X, y)
# convert to strings
X, y = to_string(X, y, n_numbers, largest)
print(X, y)
# integer encode
alphabet = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '+', ' ']
X, y = integer_encode(X, y, alphabet)
print(X, y)
# one hot encode
X, y = one_hot_encode(X, y, len(alphabet))
print(X, y)

[[3, 10]] [13]
[' 3+10'] ['13']
[[11, 3, 10, 1, 0]] [[1, 3]]
[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]] [[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]]]


In [4]:
# generate an encoded dataset
def generate_data(n_samples, n_numbers, largest, alphabet):
# generate pairs
    X, y = random_sum_pairs(n_samples, n_numbers, largest)
    # convert to strings
    X, y = to_string(X, y, n_numbers, largest)
    # integer encode
    X, y = integer_encode(X, y, alphabet)
    # one hot encode
    X, y = one_hot_encode(X, y, len(alphabet))
    # return as numpy arrays
    X, y = array(X), array(y)
    return X, y


In [12]:
# define dataset
seed(1)
n_samples = 1
n_numbers = 2
largest = 10
alphabet = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '+', ' ']
n_chars = len(alphabet)
n_in_seq_length = n_numbers * ceil(log10(largest+1)) + n_numbers - 1
n_out_seq_length = ceil(log10(n_numbers * (largest+1)))
# define LSTM configuration
n_batch = 10
n_epoch = 10
# create LSTM
model = Sequential()
model.add(LSTM(100, input_shape=(n_in_seq_length, n_chars),name='Incoder'))
model.add(RepeatVector(n_out_seq_length))
model.add(LSTM(50, return_sequences=True,name='Decoder'))
model.add(TimeDistributed(Dense(n_chars, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
# train LSTM
for i in range(n_epoch):
    X, y = generate_data(n_samples, n_numbers, largest, alphabet)
    print([invert(x, alphabet) for x in X],[invert(x, alphabet) for x in y])
    model.fit(X, y, epochs=5, batch_size=n_batch)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Incoder (LSTM)               (None, 100)               45200     
_________________________________________________________________
repeat_vector_5 (RepeatVecto (None, 2, 100)            0         
_________________________________________________________________
Decoder (LSTM)               (None, 2, 50)             30200     
_________________________________________________________________
time_distributed_5 (TimeDist (None, 2, 12)             612       
Total params: 76,012
Trainable params: 76,012
Non-trainable params: 0
_________________________________________________________________
None
[' 3+10'] ['13']
Epoch 1/5
1/1 [==============================] - 1s 1s/step - loss: 2.4814 - acc: 0.0000e+00
Epoch 2/5
1/1 [==============================] - 0s 16ms/step - loss: 2.4725 - acc: 0.5000
Epoch 3/5
1/1 [==============================] - 0s 7ms/step - l

In [11]:
expected=[]
predicted=[]
# evaluate on some new patterns
for i in range(100):
    X, y = generate_data(n_samples, n_numbers, largest, alphabet)
    result = model.predict(X, batch_size=n_batch, verbose=0)
        # calculate error
    expected.append([invert(x, alphabet) for x in y]) #= [invert(x, alphabet) for x in y]
    predicted.append([invert(x, alphabet) for x in result]) #= [invert(x, alphabet) for x in result]
# show some examples
for i in range(100):
    print('Expected=%s, Predicted=%s' % (expected[i], predicted[i]))

Expected=['17', '13', ' 6', ' 7', ' 6', ' 4', '10', '10', '15', ' 4'], Predicted=['11', '11', '11', '11', '11', '11', '11', '11', '11', '11']
Expected=['10', '14', '18', '13', '10', '11', ' 8', '14', '17', '12'], Predicted=['11', '11', '11', '11', '11', '11', '11', '11', '11', '11']
Expected=[' 9', '14', ' 9', '16', '12', ' 4', '10', '13', '13', ' 9'], Predicted=['11', '11', '11', '11', '11', '11', '11', '11', '11', '11']
Expected=[' 9', '15', '17', '13', ' 3', ' 5', ' 6', '13', '11', '19'], Predicted=['11', '11', '11', '11', '11', '11', '11', '11', '11', '11']
Expected=['11', '12', ' 7', '14', '11', '19', ' 8', ' 8', ' 9', ' 6'], Predicted=['11', '11', '11', '11', '11', '11', '11', '11', '11', '11']
Expected=[' 8', '20', ' 9', '19', '14', ' 7', '11', '19', '10', ' 7'], Predicted=['11', '11', '11', '11', '11', '11', '11', '11', '11', '11']
Expected=[' 6', '11', ' 3', ' 9', ' 5', '14', '10', '10', ' 7', ' 5'], Predicted=['11', '11', '11', '11', '11', '11', '11', '11', '11', '11']
Expect

In [ ]:
result